In [1]:
from river import anomaly
from river.anomaly import OneClassSVM
from river import feature_extraction as fx

from skmultiflow.data import FileStream             # create stream from file
from sklearn.model_selection import ParameterGrid   # hyperparameter combinations

from prequential import run_prequential

In [2]:
data_loader = FileStream(filepath='../merged_cesnet.csv')

In [3]:
ref_sample, _ = data_loader.next_sample(50)
data_loader.reset()

In [4]:
parameters = {
    'q': [0.4, 0.5, 0.6, 0.7, 0.8],
    'gamma': [0.0001, 0.001, 0.01, 0.1, 1],
    'nu': [0.02, 0.05, 0.1, 0.5]
}

svm_grid = ParameterGrid(parameters)

In [5]:
best_params = None
best_score = 0
best_metrics = []

results = []

for params in svm_grid:
    print(f"Evaluating parameters: {params}")
    data_loader.restart()

    svm_model = (
        fx.RBFSampler(gamma=params['gamma']) | OneClassSVM(nu=params['nu'])
    )

    svm_classifier = anomaly.QuantileFilter(
        svm_model,
        q=params['q']
    )

    # tracemalloc.start()

    [accuracy, precision, recall, f1], avg_processing_time, drift_idx_list = run_prequential(
        classifier=svm_classifier,
        stream=data_loader,
        feature_selector = None,
        preq_samples=50000
    )
    # current_mem, peak_mem = tracemalloc.get_traced_memory()
    # tracemalloc.stop()

    print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}")
    print(f"Average processing time: {avg_processing_time}")
    # print(f"Current memory usage (KB): {current_mem / 1024}")
    # print(f"Peak memory usage (KB): {peak_mem / 1024}")

    avg_score = (accuracy + precision + recall + f1) / 4

    if avg_score > best_score:
        best_params = params
        best_score = avg_score
        best_metrics = [accuracy, precision, recall, f1]
    
    results.append({
        **params,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'avg_score': avg_score,
        'avg_processing_time': avg_processing_time
    })

print("Grid Search Completed.")
print(f"Best Parameters: {best_params}")
print(f"Best Metrics: {best_metrics}")

Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.4}
EVALUATION: Model pretrained on 200 samples.
Accuracy: 0.689780, Precision: 0.565933, Recall: 0.996238, F1: 0.721821
Average processing time: 0.006321586618000028
Evaluating parameters: {'gamma': 0.0001, 'nu': 0.02, 'q': 0.5}
EVALUATION: Model pretrained on 200 samples.


KeyboardInterrupt: 

Accuracy: 0.718960, Precision: 0.590300, Recall: 0.994802, F1: 0.740939
Average processing time: 0.006603266086000066

In [ ]:
import pandas as pd
results = pd.DataFrame(results)
results.to_csv('oneclass.csv', index=False)